<div class="licence">
<span>Licence CC BY-NC-ND</span>
<span>François Rechenmann &amp; Thierry Parmentelat</span>
<span><img src="media/inria-25.png" /></span>
</div>

# Recherches dans une chaine

Pour rechercher un triplet dans une chaine, il existe en python un grand nombre de techniques. Commençons par la plus simple, même si elle ne nous sera pas utile directement. On commence par se donner une chaine&nbsp;:

In [ ]:
# une grande chaine
string = "GTGGCCACCGTCCTCTCTGCCAUGCCCGCCAAAATCACAUGCAACCA"

Et une sous-chaine à rechercher&nbsp;:

In [ ]:
start_codon = "AUG"

Mais n'oublions pas notre cellule usuelle&nbsp;:

In [ ]:
# la formule magique pour utiliser print() en python2 et python3
from __future__ import print_function
# pour que la division se comporte en python2 comme en python3
from __future__ import division

### Test de présence: `substring in string`

La méthode la plus simple pour savoir si une sous-chaine est présente ou non dans la chaine est celle-ci:

In [ ]:
# réponse booléenne (True ou False)
start_codon in string

### À quel(s) endroit(s): la méthode `find`

Bon, naturellement, ceci ne nous est pas très utile, car nous avons besoin de savoir à quel endroit (à quel indice) la sous-chaine est présente dans la chaine de départ. Pour cela on peut recourir à la méthode `find` sur les chaines, comme ceci&nbsp;:

In [ ]:
string.find(start_codon)

C'est déjà un peu mieux. On confirme en effet que le triplet de START est bien présent à cet endroit-là&nbsp;:

In [ ]:
string[21:24]

##### Continuer la recherche

Mais ce n'est pas le seul endroit, en effet il est aussi présent un peu plus loin. On peut naturellement relancer la recherche un peu plus loin&nbsp;:

In [ ]:
# cela fonctionne mais n'est pas optimal
string[24:].find(start_codon)

**On préfèrera toutefois**, pour des raisons de performance, utiliser la méthode `find` **sur la chaine principale**, avec un second argument qui indique où commencer la recherche; de cette façon on n'a pas besoin de créer une seconde chaine, ce qui nous donne&nbsp;:

In [ ]:
# il vaut mieux procéder comme ceci
string.find(start_codon, 24)

Ce qui nous indique qu'une deuxième occurrence est présente à l'indice `38` - qui correspond bien, on avait trouvé tout à l'heure la chaine à l'indice `14` à partir de l'indice `24` - et en effet&nbsp;:

In [ ]:
# pour vérifier
string[38:41]

##### En cas d'absence

Notez bien qu'au cas où la sous-chaine n'est pas présente, la méthode `find` renvoie `-1`. Ainsi par exemple&nbsp;:

In [ ]:
string.find('needle')

### Chercher plusieurs chaines&nbsp;: les expressions régulières

On a vu que la séquence de fin peut être encodée avec plusieurs codons différents. On pressent que la technique qu'on vient de voir ne s'y prête pas bien, et c'est pourquoi dans ce genre de cas, on utilise volontiers la notion d'**expression régulière**.

Une expression régulière est un objet mathématique qui décrit **une famille de mots** qui partagent certaines caractéristiques communes. C'est un mécanisme très puissant, qui est bien connu et utilisé dans de très nombreux contextes. 

En python, les expressions régulières sont disponibles au travers d'un module de la librairie standard qui s'appelle `re` (pour `r`egular `e`xpression). Une présentation complète de ces fonctionnalités dépasse très largement le sujet de ce cours, mais nous allons effleurer le sujet, et voir ce qui peut nous être utile dans le contexte de ce cours.

##### Un exemple simple

Et pour cela commençons comme toujours par un exemple simple&nbsp;:

In [ ]:
import re

# on appelle re.compile pour créer un objet
# de type expression régulière
a_consecutifs = re.compile("A+")

Ici on s'intéresse aux **suites de caractères `A` consécutifs**; c'est la signification de la chaine `A+`, nous allons y revenir. Ceci a pour effet de créer un objet capable de rechercher de telles séquences dans un texte. Voici comment on peut ensuite utiliser cet objet&nbsp;:

In [ ]:
# en partant d'un texte
text = "CGUCGAAAUCGAACGUAGCUCUUAAAACGCUCUGAGCGCUGACGTCGTUAG"

# on peut chercher toutes les occurrences de l'expression régulière
a_consecutifs.findall(text)

Et en effet si vous observez le `text` attentivement vous constaterez qu'il y a bien 7 suites de caractères `A` consécutifs, et avec ces longueurs-là respectivement.

##### Notre cas d'usage: chercher plusieurs chaines

Pour rechercher plusieurs chaines, voici comment faire une expression régulière qui recherche les trois codons STOP (pour rappel&nbsp;: `UAA`, `UAG`, et `UGA`)&nbsp;:

In [ ]:
# pour chercher 'UAA' ou 'UAG' ou 'UGA', on utilise le ou logique |
re_stop = re.compile("UAA|UAG|UGA")

Ce qui nous permet de trouver dans notre chaine de départ&nbsp;:

In [ ]:
re_stop.findall(text)

### Comment construire une expression régulière

On voit bien dans les exemples que tout l'art des expressions régulières consiste à élaborer la spécification de ce qui doit être reconnu ou non - dans nos deux exemples, `A+` ou `UAA|UAG|UGA` resp. On utilise pour cela le terme de motif, ou *pattern* en anglais.

##### Les briques de base

Il n'est pas question ici de couvrir tout l'éventail des possibilités, mais voici une très rapide introduction à ce mini-langage. Au départ il y a les quatre *opérateurs* `*`, `+`, `|` et `()`, qui fonctionnent comme ceci&nbsp;:

| Motif | Signification                         |
|----------|---------------------------------------|
| `ABC`    | le texte `ABC`                        |
| `A*`     | une suite *éventuellement vide* de `A`  |
| `A+`     | une suite *d'au moins un* `A`           |
| `AB`&#124;`CD`  | le texte `AB` ou le texte `CD`        |
| `AB+`    | un `A` suivi d'au moins un `B`        |
|  `(AB)*` | une suite éventuellement vide de `AB` |

##### Combiner les constructions

On peut combiner et imbriquer tous ces mécanismes. Par exemple pour décrire que l'on cherche une chaine qui contient

* d'abord: une suite d'au moins un `A`,  ou bien la chaine `CG`, 
* puis 
  * ou bien `AAA`, 
  * ou bien 1 `G` suivi de `A` ou `C`, puis au moins un `U`,
* en enfin 0 ou plus `U`.

On écrirait&nbsp;:

In [ ]:
twisted_search = re.compile("(A+|CG)(AAA|G(A|C)U+)U*")

twisted_search.findall(text)

Les débutants auront intérêt à mettre plutôt trop de parenthèses que pas assez, lorsque tous ces opérateurs sont imbriqués.

À nouveau cette liste ne fait que gratter la surface des possibilités, je laisse les plus enthousiastes d'entre vous approfondir le sujet par eux mêmes ou sur le forum.

### Comment utiliser une expression régulière

Avec la méthode `findall` que nous avons jusqu'ici utilisée sur les regexps, nous trouvons bien les **contenus** des chaines recherchées, mais à nouveau **pas leurs positions**.

Pour accéder à ces détails, il faut utiliser un autre type d'objet, que nous allons appeler un *match*, et qui va contenir davantage de détails sur l'occurrence où on a trouvé l'expression régulière. 

##### La méthode `search`

Un objet *match* se présente comme ceci&nbsp;:

In [ ]:
# cherchons la première occurrence de STOP dans `text`
match = re_stop.search(text)

Si on examinait le résultat on verrait un objet python, de type `SRE_Match`, qui regroupe tous les détails qui nous intéressent, et notamment&nbsp;:

In [ ]:
# la sous-chaine qui a correspondu à la recherche
print("correspondance", match.group())
# l'indice de début de l'expression régulière dans la chaine
print("indice de début", match.start())
# l'indice de fin de l'expression régulière dans la chaine
print("indice de fin", match.end())

Comme pour la méthode `find` que l'on avait vue pour la recherche simple, on peut évidemment rechercher notre expression à partir d'un certain indice dans le texte de départ&nbsp;:

In [ ]:
# commencer la recherche à l'indice 20
match2 = re_stop.search(text, 20)

Si on prend maintenant le temps d'écrire une petite fonction pour montrer un *match*, et en tirant profit de [davantage d'informations qui sont disponibles dans un objet *match*](https://docs.python.org/2/library/re.html#match-objects)&nbsp;:

In [ ]:
# pour montrer une occurrence de recherche
def afficher_match(match):
    print("pattern {} à partir de {}".
          format(match.re.pattern, match.pos), end="")
    print(" -> trouvé {} entre {} et {}".
          format(match.group(), match.start(), match.end()))

In [ ]:
# ce qui donne avec notre deuxieme recherche
afficher_match(match2)

Notons enfin que dans le cas où l'expression régulière ne peut pas être trouvée, `search` retourne `None`.

##### La méthode `finditer`

Les objets de type *expression régulière* proposent également une méthode très utile qui s'appelle `finditer`, qui permet d'itérer sur toutes les occurrences de la recherche.

In [ ]:
# pour inspecter toutes les occurrences en une simple passe
for match in re_stop.finditer(text):
    afficher_match(match)

Sans entrer dans les détails, le type d'objet renvoyé par la méthode `finditer` s'appelle en python un *itérable*, c'est-à-dire un objet sur lequel on peut écrire une boucle `for`&nbsp;; d'où la provenance du nom.

### Pour en savoir plus

Pour les curieux, je vous invite à vous pencher sur ces documents qui font partie de la documentation de la librairie standard python&nbsp;:

* [la documentation python sur le module `string`](https://docs.python.org/2/library/string.html)
* [la documentation python sur le module `re` - les expressions régulières (utilisateurs avancés)](https://docs.python.org/2/library/re.html)
* ainsi que de très nombreux tutoriaux sur les regexps sur la toile...

Je signale également des sites web permettant de tester et mettre au point ses expressions régulières, comme notamment

* http://pythex.org/
* https://regex101.com/
qui peuvent être pratiques si vous avez une expression un peu lourde et compliquée.